In [23]:
import nltk
from textblob import TextBlob as tb
import math
import io
from __future__ import division
import numpy
import pandas as pd
import matplotlib
import nltk
import re
import os
import string
import codecs
from sklearn import feature_extraction
import mpld3
%matplotlib inline 

from sklearn.feature_extraction.text import TfidfVectorizer

os.chdir('/users/asheets/Documents/Work_Computer_new/Work_Computer/Grad_School/PREDICT_453/Notebooks/DSI/')

all_docs = []
dsis = []
d = 1

cachedStopWords = nltk.corpus.stopwords.words('english')
pattern = re.compile(r'\b(' + r'|'.join(cachedStopWords) + r')\b\s*')

for i in range(40):
    doc_name = 'DSI' + str(d) + '.txt'
    try:
        with open(doc_name, 'r') as f:
            sample = f.read()
        sample = sample.decode('utf-8')
       #sample = sample.decode('ascii')
        sample = sample.lower()
        sample = re.sub(r'[^\w]', ' ', sample)
        sample = ''.join([i for i in sample if not i.isdigit()])
        sample = pattern.sub('', sample)
        sample = "".join(l for l in sample if l not in string.punctuation)
        sample2 = " ".join(k for k in tb(sample).noun_phrases)
        all_docs.append(sample)
        dsis.append('DSI' + str(d))
        d = d + 1
    except IOError:
        d = d + 1
        pass


In [17]:
#Using resource: http://brandonrose.org/clustering
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens


In [18]:

from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(all_docs) #fit the vectorizer to all_docs

print(tfidf_matrix.shape)

CPU times: user 789 ms, sys: 19.5 ms, total: 809 ms
Wall time: 869 ms
(38, 318)


In [19]:
terms = tfidf_vectorizer.get_feature_names()
terms

[u'accord',
 u'act',
 u'action',
 u'ad',
 u'addit',
 u'address',
 u'afford',
 u'agenc',
 u'agre',
 u'agreement',
 u'allow',
 u'alreadi',
 u'america',
 u'american',
 u'announc',
 u'anoth',
 u'appear',
 u'approv',
 u'argu',
 u'ask',
 u'associ',
 u'attempt',
 u'author',
 u'barack',
 u'barack obama',
 u'base',
 u'begin',
 u'believ',
 u'benefit',
 u'better',
 u'big',
 u'billion',
 u'border',
 u'build',
 u'busi',
 u'c',
 u'campaign',
 u'care',
 u'case',
 u'chairman',
 u'chang',
 u'china',
 u'choic',
 u'claim',
 u'clean',
 u'clear',
 u'climat',
 u'climat chang',
 u'clinton',
 u'close',
 u'come',
 u'comment',
 u'committe',
 u'communiti',
 u'compani',
 u'competit',
 u'complet',
 u'concern',
 u'confirm',
 u'congress',
 u'congression',
 u'conserv',
 u'consid',
 u'construct',
 u'continu',
 u'control',
 u'cost',
 u'countri',
 u'court',
 u'creat',
 u'critic',
 u'current',
 u'cut',
 u'day',
 u'deal',
 u'decis',
 u'democrat',
 u'depart',
 u'develop',
 u'direct',
 u'director',
 u'discuss',
 u'domest',


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

[[ -2.22044605e-16   7.58817823e-01   7.50561818e-01 ...,   8.63109036e-01
    8.71101928e-01   9.03437621e-01]
 [  7.58817823e-01   0.00000000e+00   7.89780964e-01 ...,   8.41992185e-01
    8.03234972e-01   9.13070580e-01]
 [  7.50561818e-01   7.89780964e-01  -2.22044605e-16 ...,   7.84435237e-01
    7.61196456e-01   9.18665379e-01]
 ..., 
 [  8.63109036e-01   8.41992185e-01   7.84435237e-01 ...,  -2.22044605e-16
    9.01100246e-01   8.64179997e-01]
 [  8.71101928e-01   8.03234972e-01   7.61196456e-01 ...,   9.01100246e-01
    0.00000000e+00   9.52677042e-01]
 [  9.03437621e-01   9.13070580e-01   9.18665379e-01 ...,   8.64179997e-01
    9.52677042e-01   0.00000000e+00]]


In [22]:
#kmeans
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 181 ms, sys: 10.4 ms, total: 192 ms
Wall time: 232 ms


In [24]:
DSIs = { 'DSI': dsis, 'doc': all_docs, 'cluster': clusters }

frame = pd.DataFrame(DSIs, index = [clusters] , columns = ['DSI', 'cluster'])
frame

,DSI,cluster
1,DSI1,1
4,DSI2,4
3,DSI3,3
1,DSI4,1
3,DSI5,3
1,DSI6,1
0,DSI7,0
0,DSI8,0
2,DSI9,2
4,DSI10,4
